Data Source:
"Common Core of Data School District Finance Survey (F-33), FY 2020." National Center for Education Statistics, U.S. Department of Education. 
Accessed from: 
- [NCES Website – CCD Data Files](https://nces.ed.gov/ccd/files.asp#Fiscal:1,Page:1)
- [NCES Website – Shape Files](https://nces.ed.gov/programs/edge/Geographic/SchoolLocations)
- [Zip File Download from NCES](https://nces.ed.gov/programs/edge/data/EDGE_GEOCODE_PUBLICSCH_1920.zip)


# Local Education Agency Finance Survey – School District Data 2019 – 2020  
Local Education Agency will be abbreviated as LEA

In [64]:
# If you are running this code on your local machine and do not have necessary packages installed,
# Uncomment the packages you need and run this cell first. 
# Once installed, replace the comment and proceed with running the remainder of the notebook. 

#!pip install --upgrade pip
#!pip install pandas
#!pip install numpy
#!pip install seaborn
#!pip install matplotlib
#!pip install sqlalchemy
#!pip install pandas sqlalchemy psycopg2-binary
#!pip install scikit-lean

### Import Packages

In [65]:
import pandas as pd
import numpy as np
import os
import json
from sqlalchemy import create_engine
import seaborn as sns # import is for upcoming use
import matplotlib.pyplot as plt # import is for upcoming use
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

### Importing the Dataset

When importing the dataset, follow these steps for best practices and to ensure accuracy:

1. **Locate the Dataset:**
   Ensure that the dataset file is present in the project directory. This is where the import function will look for the file.

2. **Understand the File Format:**
   Our dataset is in a TAB-delimited format. When using `pandas.read_csv` or similar functions, specify the delimiter with `delimiter='\t'` to correctly parse the file.

3. **Verify the Import:**
   After importing, it's crucial to do a quick check of the DataFrame:
   - Use `df.head()` to preview the first few rows.

```python
df = pd.read_csv('sdf20_1a.txt', delimiter= '\t')

# Preview the first few rows of the DataFrame
df.head()

In [66]:
file_name = 'sdf18.txt'
file_path = os.path.join(".", "raw_data_files", file_name)

df = pd.read_csv(file_path, delimiter= '\t')

df.head()

<ipython-input-66-db161e859e94>:4: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, delimiter= '\t')


,LEAID,CENSUSID,FIPST,CONUM,CSA,CBSA,NAME,STNAME,STABBR,SCHLEV,...,FL_66V,FL_W01,FL_W31,FL_W61,FL_V95,FL_V02,FL_K14,FL_CE1,FL_CE2,FL_CE3
0,100002,N,1,1073,142,13820,Alabama Youth Services,Alabama,AL,N,...,M,M,M,M,M,M,M,M,M,M
1,100005,01504840100000,1,1095,290,10700,Albertville City,Alabama,AL,03,...,M,R,R,R,R,M,M,M,M,M
2,100006,01504800100000,1,1095,290,10700,Marshall County,Alabama,AL,03,...,M,R,R,R,R,M,M,M,M,M
3,100007,01503740100000,1,1073,142,13820,Hoover City,Alabama,AL,03,...,M,R,R,R,R,M,M,M,M,M
4,100008,01504530100000,1,1089,290,26620,Madison City,Alabama,AL,03,...,M,R,R,R,R,M,M,M,M,M


### Import the Column Mapping
I prepared an excel file that has the original column names, the new names of the columns, their expected datatype in a database, and the description. This file will serve as a quick and easy way to map the new columns with less code, and maintaining a dictionary of the columns.

In [67]:
# Import Column Map
column_mapping_df = pd.read_excel('LEA Local Finance Survey – School District Data 2019 – 2020 – Column Mapping.xlsx', 
                                  sheet_name='Column Mapping')

# Remove White Spaces from Column Names
column_mapping_df['Original Name'] = column_mapping_df['Original Name'].str.strip()
column_mapping_df['New Name'] = column_mapping_df['New Name'].str.strip()

column_mapping_df.head()

,Original Name,New Name,Type,Table,Description
0,LEAID,lea_id,VARCHAR(7),entity,National Center For Education Statistics (NCES...
1,CENSUSID,census_id,VARCHAR(14),all,Census Bureau 14-Digit Government Id
2,FIPST,ansi_state_code,VARCHAR(2),entity,American National Standards Institute (ANSI) S...
3,CONUM,ansi_county_code,VARCHAR(7),entity,American National Standards Institute (ANSI) C...
4,CSA,csa,VARCHAR(3),entity,Consolidated Statistical Area


In [68]:
# Create Dictionary to Map New Column Names
column_map_dict = column_mapping_df.set_index('Original Name')['New Name'].to_dict()

# Rename Columns
df.rename(columns=column_map_dict, inplace=True)
df.columns = df.columns.str.strip()

In [69]:
df.dtypes

lea_id                                      object
census_id                                   object
ansi_state_code                              int64
ansi_county_code                            object
csa                                         object
                                             ...  
tech_related_supplies_services_flag         object
tech_related_equipment_flag                 object
curr_expenditures_state_local_funds_flag    object
curr_expenditures_federal_funds_flag        object
curr_expenditures_resa_lea_flag             object
Length: 262, dtype: object

After renaming the columns, I used `df.dtypes` to confirm the names of the columns were correct, but also so I can get an idea of what columns may need cleaning to acheive a certain data type.

## Exclusion of Non-Government Entities from Analysis

The Census Bureau has specific criteria to determine if a Local Education Agency (LEA) qualifies as a government entity. These criteria include the LEA's power to:

- Levy taxes
- Independently manage its own budget
- Appoint its school board members without oversight from other local government bodies

An LEA that satisfies these conditions is considered a government entity and is assigned a unique `census_id`. This identifier signals eligibility for federal, state, and local funding, which is often dependent on an LEA's tax authority and fiscal independence.

However, LEAs that do not meet these criteria are assigned an 'N' for their `census_id`. This indicates that they are not recognized as government entities by the Census Bureau and, consequently, are not typically eligible for the tax-based funding that our analysis focuses on. Therefore, these LEAs are excluded from our dataset to maintain a focus on entities eligible for such funding.

By removing rows where `census_id` is 'N', we ensure that our analysis only includes LEAs that have the potential to receive and manage federal, state, and local funding in line with our research objectives.


In [70]:
df = df[df['census_id'] != 'N']

In [71]:
df['census_id'].duplicated().any()

False

After removing the Census IDs that had the 'N' placeholder, I wanted to confirm that there were no duplicate Census IDs. This is in preparation for this being the Primary Key within the database table keeping the LEA Entity information. This will serve as a Foreign Key in subsequent tables to link records to the Entity.  
#### Expected Result : False

## Column Removal for Database Normalization

As part of the data normalization process for database insertion, we target columns starting with 'total_' for removal. These columns are presumed to contain aggregate data that may not be suitable for the normalized database structure. Prior to their removal, the content of these columns is preserved by transferring it to a separate DataFrame. This precaution ensures that the aggregate data remains accessible for any future analysis or reference requirements.


In [72]:
total_columns = []

for col in df.columns: 
    if col.startswith('total_'):
        total_columns.append(col)

total_columns

['total_revenue',
 'total_federal_revenue',
 'total_state_revenue',
 'total_local_revenue',
 'total_expenditures',
 'total_curr_expenditures_pri_sec_ed',
 'total_curr_expenditures_instruction',
 'total_curr_expenditures_support_services',
 'total_current_expenditures_other_prim_sec',
 'total_non_prim_sec_expenditures',
 'total_capital_outlay_expenditures',
 'total_salaries',
 'total_employee_benefits',
 'total_salaries_flag',
 'total_employee_benefits_flag']

In [73]:
column_totals = df[total_columns].copy()
column_totals

,total_revenue,total_federal_revenue,total_state_revenue,total_local_revenue,total_expenditures,total_curr_expenditures_pri_sec_ed,total_curr_expenditures_instruction,total_curr_expenditures_support_services,total_current_expenditures_other_prim_sec,total_non_prim_sec_expenditures,total_capital_outlay_expenditures,total_salaries,total_employee_benefits,total_salaries_flag,total_employee_benefits_flag
1,56909000,7691000,33512000,15706000,50832000,47501000,27292000,16507000,3702000,887000,642000,25623000,10362000,R,R
2,59036000,7843000,37538000,13655000,61817000,57363000,30466000,22741000,4156000,663000,3074000,31494000,12821000,R,R
3,179516000,6516000,72905000,100095000,178534000,155621000,94966000,52622000,8033000,3102000,12432000,90022000,35382000,R,R
4,119390000,5858000,64240000,49292000,116231000,101727000,61437000,35594000,4696000,1175000,8932000,56669000,21651000,R,R
6,20007000,1544000,11917000,6546000,20693000,19000000,10480000,7417000,1103000,310000,177000,10630000,4118000,R,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18704,76124000,1926000,23021000,51177000,74341000,52554000,32616000,18580000,1358000,0,21371000,31301000,14274000,R,R
18705,6657000,175000,4983000,1499000,6384000,5758000,3113000,2436000,209000,69000,557000,3115000,1539000,R,R
18706,25414000,1822000,18300000,5292000,24575000,22545000,14312000,7415000,818000,0,889000,12900000,5907000,R,R
18707,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,N,N


In [74]:
df.drop(columns= total_columns, inplace= True)

In [75]:
df['year'] = df['year'].astype(str)
df['year'] = '20' + df['year']
df['year']

1        2018
2        2018
3        2018
4        2018
6        2018
         ... 
18704    2018
18705    2018
18706    2018
18707    2018
18708    2018
Name: year, Length: 14625, dtype: object

## Casting Data Types
In the above cell, I am converting year to a String so I can add '20' to the year in order to have the correct format to convert to datetime.
In the below cells:
- ansi_state_code and ansi_county_code are converted to Strings because these will not be aggregated at any point. 
- year is being converted to datetime.
- ccd_nonfiscal_match and census_fiscal_match are being converted to booleans to match database data type requirements.

In [76]:
df['ansi_state_code'] = df['ansi_state_code'].astype(str)
df['ansi_county_code'] = df['ansi_county_code'].astype(str)
df['year'] = pd.to_datetime(df['year'].astype(str), format='%Y')
df['ccd_nonfiscal_match'] = df['ccd_nonfiscal_match'].astype(bool)
df['census_fiscal_match'] = df['census_fiscal_match'].astype(bool)

In [77]:
df[['ansi_state_code', 'ansi_county_code', 'year', 'ccd_nonfiscal_match', 'census_fiscal_match']].dtypes

ansi_state_code                object
ansi_county_code               object
year                   datetime64[ns]
ccd_nonfiscal_match              bool
census_fiscal_match              bool
dtype: object

In [78]:
df.describe(include='all')

,lea_id,census_id,ansi_state_code,ansi_county_code,csa,cbsa,lea_name,state,st_abbr,school_level_code,...,short_term_debt_outstanding_end_fisc_year_flag,assets_sinking_fund_flag,assets_bond_fund_flag,assets_other_funds_flag,utilities_services_flag,tech_related_supplies_services_flag,tech_related_equipment_flag,curr_expenditures_state_local_funds_flag,curr_expenditures_federal_funds_flag,curr_expenditures_resa_lea_flag
count,14625.0,14625,14625,14625,14625,14625,14625,14625,14625,14625,...,14625,14625,14625,14625,14625,14625,14625,14625,14625,14625
unique,14625.0,14625,51,3125,174,932,14318,51,51,7,...,3,4,5,5,5,3,4,4,4,4
top,100005.0,01504840100000,6,17031,N,N,Jefferson County,California,CA,03,...,R,R,R,R,R,R,R,R,R,M
freq,1.0,1,1115,165,6953,3817,5,1115,1115,10473,...,11135,13228,13487,13617,9501,8213,7865,6981,7374,11456
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Data Cleaning Notes

**Handling Special Placeholders in Financial Data:**

The dataset uses special placeholder values to indicate non-standard entries for financial data: 
- “-1” indicates missing data, which may arise in situations where zero values are ambiguous.
- “-2” and “-3” could similarly indicate other forms of non-standard or suppressed data, such as revised figures or privacy-related omissions.

To facilitate accurate analysis, we replace these placeholder values in the money-related fields to avoid distortions in statistical calculations. However, each financial field is paired with a corresponding "flag" column. These flag columns provide references to documentation that explain the classification of each value in more depth, including the placeholders.

The purpose of this cleaning step is not to discard the nuances and details encoded by these placeholders but to create a dataset that can be analyzed quantitatively without misinterpretation caused by non-numeric values. The flag columns remain intact for any case-by-case examination where the context behind the numeric values is necessary, ensuring transparency and traceability in our dataset.

This approach ensures that while the dataset is primed for quantitative analysis, the integrity and comprehensiveness of the data are maintained for more qualitative assessments.


In [79]:
df.replace(-3, np.nan, inplace=True)
df.replace(-2, np.nan, inplace=True)
df.replace(-1, np.nan, inplace=True)

In [80]:
df.describe()

,year,fall_membership,fall_membership_school_univ,title_I_thru_state,indiv_with_disabilities_thru_state,C16,C17,voc_tech_education_thru_state,bilingual_education_thru_state,other_thru_state,...,assets_sinking_fund,assets_bond_fund,assets_other_funds,utilities_services,tech_related_supplies_services,tech_related_equipment,curr_expenditures_state_local_funds,curr_expenditures_federal_funds,curr_expenditures_resa_lea,weight
count,14625,13404.000000,13754.000000,1.427200e+04,1.427200e+04,1.427200e+04,1.427200e+04,1.427200e+04,1.427200e+04,1.427200e+04,...,1.427200e+04,1.427200e+04,1.427200e+04,1.427200e+04,1.427200e+04,1.427200e+04,7.985000e+03,7.985000e+03,1.427200e+04,14625.0
mean,2018-01-01 00:00:00,3622.945166,3520.507198,9.656714e+05,7.809238e+05,8.757560e+04,5.664588e+03,3.772085e+04,2.307371e+04,4.363446e+05,...,1.684007e+06,5.207361e+06,1.148048e+07,6.013274e+05,3.150437e+05,9.595011e+04,3.452151e+07,3.232660e+06,1.088165e+05,1.0
min,2018-01-01 00:00:00,0.000000,-9.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0
25%,2018-01-01 00:00:00,402.000000,367.000000,5.800000e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.000000e+03,...,0.000000e+00,0.000000e+00,1.137000e+06,0.000000e+00,0.000000e+00,0.000000e+00,4.036000e+06,2.740000e+05,0.000000e+00,1.0
50%,2018-01-01 00:00:00,1107.000000,1053.500000,1.890000e+05,1.380000e+05,8.000000e+03,0.000000e+00,0.000000e+00,0.000000e+00,5.000000e+04,...,6.000000e+03,0.000000e+00,3.424500e+06,1.100000e+05,0.000000e+00,0.000000e+00,1.026600e+07,7.230000e+05,0.000000e+00,1.0
75%,2018-01-01 00:00:00,2964.000000,2870.000000,5.672500e+05,5.670000e+05,5.500000e+04,0.000000e+00,1.300000e+04,0.000000e+00,2.180000e+05,...,6.730000e+05,9.200000e+05,9.233250e+06,4.410000e+05,1.200000e+05,3.000000e+03,2.691800e+07,2.080000e+06,0.000000e+00,1.0
max,2018-01-01 00:00:00,976771.000000,962949.000000,5.881240e+08,1.420570e+08,3.151900e+07,4.726000e+06,1.000600e+07,1.418600e+07,1.588090e+08,...,8.852660e+08,1.591297e+09,3.464358e+09,8.521400e+07,1.564930e+08,2.450700e+07,2.928478e+09,4.574960e+08,4.446180e+08,1.0
std,NaN,14781.233586,14517.280160,7.282620e+06,3.116454e+06,5.026265e+05,6.879561e+04,1.915870e+05,1.985518e+05,2.561761e+06,...,1.185297e+07,2.959688e+07,4.346642e+07,2.374472e+06,2.257338e+06,6.698228e+05,1.133112e+08,1.357562e+07,3.778499e+06,0.0


### Entity Schema Tables  

#### Create entity DataFrame

In [81]:
# Initialize an empty list for storing column names
entity_columns = []

# Iterate over each row in the mapping DataFrame
for index, row in column_mapping_df.iterrows():
    # Check if the table is 'entity' or 'all', and the column name is not 'year'
    if row['Table'] in ['entity', 'all'] and row['New Name'] != 'year':
        if row['New Name'] not in total_columns:
            # Add the new column name to the list
            entity_columns.append(row['New Name'])

# Create a new DataFrame with only the selected columns
entity = df[entity_columns].copy()
entity


,lea_id,census_id,ansi_state_code,ansi_county_code,csa,cbsa,lea_name,state,st_abbr
1,100005,01504840100000,1,1095,290,10700,Albertville City,Alabama,AL
2,100006,01504800100000,1,1095,290,10700,Marshall County,Alabama,AL
3,100007,01503740100000,1,1073,142,13820,Hoover City,Alabama,AL
4,100008,01504530100000,1,1089,290,26620,Madison City,Alabama,AL
6,100011,01503710100000,1,1073,142,13820,Leeds City,Alabama,AL
...,...,...,...,...,...,...,...,...,...
18704,5605830,51502000200000,56,56039,N,27220,Teton County School District #1,Wyoming,WY
18705,5606090,51502300200000,56,56045,N,N,Weston County School District #7,Wyoming,WY
18706,5606240,51502200400000,56,56043,N,N,Washakie County School District #1,Wyoming,WY
18707,5680180,51500340100000,56,56005,N,23940,Northeast Wyoming BOCES,Wyoming,WY


#### Create annual_stats DataFrame

In [82]:
# Initialize an empty list for storing column names
annual_stats_columns = []

# Iterate over each row in the mapping DataFrame
for index, row in column_mapping_df.iterrows():
    # Check if the table is 'annual_stats' or 'all'
    if row['Table'] in ['annual_stats', 'all']:
        if row['New Name'] not in total_columns:
            # Add the new column name to the list
            annual_stats_columns.append(row['New Name'])

# Create a new DataFrame with only the selected columns
annual_stats = df[annual_stats_columns].copy()

# If 'year' is not the last column, move it to the end
if 'year' in annual_stats.columns and annual_stats.columns[-1] != 'year':
    # Get a list of all columns except 'year'
    cols = [col for col in annual_stats.columns if col != 'year']
    # Add 'year' at the end of the list
    cols.append('year')
    # Reorder the DataFrame
    annual_stats = annual_stats[cols]

annual_stats

,census_id,school_level_code,agency_charter_code,ccd_nonfiscal_match,census_fiscal_match,low_grade_offered,high_grade_offered,fall_membership,fall_membership_school_univ,fall_membership_flag,fall_membership_school_univ_flag,year
1,01504840100000,03,3,True,True,PK,12,5562.0,5562.0,R,R,2018-01-01
2,01504800100000,03,3,True,True,PK,12,5662.0,5662.0,R,R,2018-01-01
3,01503740100000,03,3,True,True,PK,12,14027.0,14027.0,R,R,2018-01-01
4,01504530100000,03,3,True,True,PK,12,10767.0,10767.0,R,R,2018-01-01
6,01503710100000,03,3,True,True,PK,12,1974.0,1974.0,R,R,2018-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...
18704,51502000200000,03,3,True,True,KG,12,2862.0,2862.0,R,R,2018-01-01
18705,51502300200000,03,3,True,True,KG,12,254.0,254.0,R,R,2018-01-01
18706,51502200400000,03,3,True,True,KG,12,1274.0,1274.0,R,R,2018-01-01
18707,51500340100000,07,3,True,True,01,12,NaN,28.0,A,R,2018-01-01


### Expenses & Revenue Schema Tables

#### `melt_df()`
The purpose of this fuction is to convert the data from a wide format to a long format, which is optimal for data normalization in relational databases, and for data visualizations.

In [83]:
def melt_df(df: pd.DataFrame, schema: str, table: str, column_mapping_df: pd.DataFrame, total_columns: list) -> pd.DataFrame:
    # Initialize an empty list for storing column names
    columns_to_use = []
    new_columns = []
    
    # Iterate over each row in the mapping DataFrame
    for _, row in column_mapping_df.iterrows():
        # Check if the table is the specified one or 'all'
        if row['Table'] in [table, 'all']:
            if row['New Name'] not in total_columns:
                # Add the new column name to the list
                columns_to_use.append(row['New Name'])
    
    # Create a new DataFrame with only the selected columns
    new_df = df[columns_to_use].copy()
    
    # Select id_vars for the melt function
    id_vars = ['census_id', 'year'] + [col for col in new_df.columns if col.endswith('_flag')]
    
    # Melt the DataFrame
    if schema == 'expenses':
        new_df = pd.melt(new_df, id_vars=id_vars, var_name='expenditure_title', value_name='amount')
        # The new columns will be 'expenditure_title' and 'amount'
        new_columns = ['expenditure_title', 'amount']
    elif schema == 'revenue':
        new_df = pd.melt(new_df, id_vars=id_vars, var_name='revenue_title', value_name='revenue')
        # The new columns will be 'revenue_title' and 'revenue'
        new_columns = ['revenue_title', 'revenue']
    
    # Ensure the new columns are at indexes 2 and 3
    # Get the list of id_vars that don't include the new columns
    remaining_columns = [col for col in id_vars if col not in new_columns]

    # Reorder columns such that new columns are at index 2 and 3
    ordered_columns = remaining_columns[:2] + new_columns + remaining_columns[2:]
    
    # Reassign the DataFrame with the ordered columns
    new_df = new_df[ordered_columns]
    
    return new_df


#### Create expenditures DF

In [84]:
expenditures = melt_df(df,'expenses', 'expenditures', column_mapping_df, total_columns)
expenditures.head()

KeyError: "['special_education_expenditure_curr', 'special_education_expenditure_instructional', 'special_education_expenditure_pupil_support', 'special_education_expenditure_instructional_staff_support', 'special_education_expenditure_student_transportation_support', 'cares_act_expenditure_curr', 'cares_act_expenditure_instructional', 'cares_act_expenditure_support_services', 'cares_act_expenditure_capital_outlay', 'cares_act_expenditure_tech_related_supplies_services', 'cares_act_expenditure_tech_related_equipment', 'special_education_expenditure_curr_flag', 'special_education_expenditure_instructional_flag', 'special_education_expenditure_pupil_support_flag', 'special_education_expenditure_instructional_staff_support_flag', 'special_education_expenditure_student_transportation_support_flag', 'cares_act_expenditure_curr_flag', 'cares_act_expenditure_instructional_flag', 'cares_act_expenditure_support_services_flag', 'cares_act_expenditure_capital_outlay_flag', 'cares_act_expenditure_

#### Create local DataFrame

In [ ]:
local = melt_df(df,'revenue', 'local_revenue', column_mapping_df, total_columns)
local.head()

#### Create state DataFrame

In [ ]:
state = melt_df(df,'revenue', 'state_revenue', column_mapping_df, total_columns)
state.head()

#### Create federal DataFrame

In [ ]:
federal = melt_df(df,'revenue', 'federal_revenue', column_mapping_df, total_columns)
federal.head()

## Create Database Mapping
The keys of the dictionary are the table names within the database.
Values:
- Index 0 = Schema Name
- Index 1 = DataFrame Name

In [ ]:
database_map = {'entity' : ['entity', entity],
                'annual_stats' : ['entity', annual_stats],
                'expenditures' : ['expenses', expenditures],
                'federal_revenue' : ['revenue', federal],
                'state_revenue' : ['revenue', state],
                'local_revenue' : ['revenue', local]}

## Database Initialization with Mapped Data
The code snippet enclosed within the conditional block is designed for the initial population of the database. As this project evolves, we will enhance this section with more sophisticated logic and additional functionality to support incremental updates and data management requirements.

In [ ]:
# use_database = input("Enter 'y' to use database script. Else enter 'n'")
use_database = 'n'
if use_database == 'y':
    
    # Read in database credentials from JSON file
    with open('LEA_Finance_Survey_DB.json') as infile:
        credentials = json.load(infile)
    
    # Assign Credentials to Variables
    database_name = credentials['database']
    username = credentials['user']
    password = credentials['password']
    host = credentials['host']
    port = credentials['port']

    # Create a database connection using SQLAlchemy engine
    engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database_name}')
    
    populate_new_tables = 'n'

    if populate_new_tables == 'y':
        # Iterate over the database_map to insert each DataFrame
        for table_name, [schema_name, df_to_export] in database_map.items():
            df_to_export.to_sql(table_name, engine, schema=schema_name, if_exists='append', index=False)

    engine.dispose()


### Data Normalization for Visualization

In preparing our dataset for visualization in Tableau, we employ normalization techniques on specific columns to ensure that our visualizations are not biased by the scale of the data:

#### Min-Max Scaling
- **Purpose**: To transform the data into a fixed range of 0 to 1, making it easier to visualize different variables on the same scale without distorting the distribution of values. This is particularly important when creating comparative visualizations, such as heatmaps or line charts, where relative scales matter.
- **Applied to**: Columns like `[revenue]`, where we need to maintain the relative distribution of the values for accurate visual comparison.

#### Z-Score Standardization
- **Purpose**: To standardize values so that they have a mean of zero and a standard deviation of one. This normalization is useful for visualizations that compare the relative standing of data points within a distribution, such as histograms or scatter plots.
- **Applied to**: Columns like `[revenue]`, which benefits from showing how many standard deviations away from the mean the data points are, thus facilitating a clear interpretation of outliers and distribution spread.

By normalizing the data before visualization, we aim to create clear and meaningful visualizations in Tableau that accurately represent the underlying data without the distortion that can come from varying scales.

In [ ]:
scaler = MinMaxScaler()

expenditures["amount (Min/Max Scale)"] = scaler.fit_transform(expenditures[["amount"]])
federal["revenue (Min/Max Scale)"] = scaler.fit_transform(federal[["revenue"]])
state["revenue (Min/Max Scale)"] = scaler.fit_transform(state[["revenue"]])
local["revenue (Min/Max Scale)"] = scaler.fit_transform(local[["revenue"]])

In [ ]:
scaler = StandardScaler()

expenditures["amount (Z-Score Std)"] = scaler.fit_transform(expenditures[["amount"]])
federal["revenue (Z-Score Std)"] = scaler.fit_transform(federal[["revenue"]])
state["revenue (Z-Score Std)"] = scaler.fit_transform(state[["revenue"]])
local["revenue (Z-Score Std)"] = scaler.fit_transform(local[["revenue"]])

### Code Commentary on Exporting DataFrames

The subsequent code snippet performs the operation of exporting DataFrames to CSV files. These CSV files include derived values such as Z-Scores and Min/Max statistics. This export facilitates further analysis in data visualization tools like Tableau. Although these derived values are excluded from the database for flexibility and to adhere to best practices, they are being included in the CSV exports specifically for the purpose of exploratory analysis outside the database environment.

In [ ]:
#entity.to_csv('LEA Finance Survey – Entity Data.csv')
#annual_stats.to_csv('LEA Finance Survey – Entity – Annual Stats Data.csv')
#expenditures.to_csv('LEA Finance Survey – Expenditures Data.csv')
#federal.to_csv('LEA Finance Survey – Federal Revenue Data.csv')
#state.to_csv('LEA Finance Survey – State Revenue Data.csv')
#local.to_csv('LEA Finance Survey – Local Revenue Data.csv')